In [1]:
import $ivy.`io.github.quafadas::scautable:0.0.33`
import $ivy.`io.github.quafadas::vecxt::0.0.34`
import $ivy.`io.github.quafadas::inspireRAD::0.0.17`
import $ivy.`org.apache.commons:commons-math3:3.6.1`
import $ivy.`org.typelevel::spire::0.18.0`
import $ivy.`io.github.quafadas::dedav4s:0.10.0-RC2`
import $ivy.`org.typelevel::cats-core::2.13.0`
import $exec.`utils2`
// heatmap plotting
import $exec.`heatmap2`
import io.github.quafadas.table.*
import viz.PlotTargets.almond

import scala.util.chaining.*
import org.apache.commons.math3.distribution.EnumeratedIntegerDistribution

import $ivy.$                                     

import $ivy.$                                  

import $ivy.$                                       

import $ivy.$                                       

import $ivy.$                             

import $ivy.$                                       

import $ivy.$                                 

import $exec.$       
// heatmap plotting

import $exec.$         

import io.github.quafadas.table.*

import viz.PlotTargets.almond


import scala.util.chaining.*

import org.apache.commons.math3.distribution.EnumeratedIntegerDistribution


This notebook attemps to rediscover the classical solution by training a (very) simple neural network. 

In [2]:
import cats.syntax.all.toShow
import vecxt.all.Matrix
import vecxt.matrixUtil.row
import vecxt.all.fromRowsArray
import vecxt.BoundsCheck.DoBoundsCheck.yes

val normalDist = new org.apache.commons.math3.distribution.NormalDistribution()
val smooth = true

val dimensions = 27 * 27

def saveMatrixToCSV(matrix: Matrix[Double], filePath: String): Unit =
  val lines = for (i <- 0 until matrix.rows) yield matrix.row(i).mkString(",")
  os.write.over(os.Path(filePath, os.pwd), lines.mkString("\n"))

def loadMatrixFromCSV(filePath: String)(using
    ClassTag[Double]
): Matrix[Double] =
  import vecxt.BoundsCheck.DoBoundsCheck.yes
  val lines = os.read.lines(os.Path(filePath, os.pwd))
  val data = lines.map(_.split(",").map(_.toDouble)).toArray
  Matrix.fromRowsArray(data)

def onehot(char: Char, allChars: collection.Map[Char, Int]): Array[Double] =
  val idx2 = allChars(char)
  Array.fill(allChars.size)(0.0).tap(_(idx2) = 1.0)


val generateWeights = true
val W = if (generateWeights)
  val randoms = Array.fill(dimensions)(normalDist.sample())
  val tmp = Matrix(randoms, (27, 27))
  saveMatrixToCSV(tmp, "weights.csv")
  tmp
else
  loadMatrixFromCSV("weights.csv")

val chars = '.' +: ('a' to 'z').toVector
val charsIndex = (0 to 26).toVector
val charsMap = chars.zip(charsIndex).toMap
val i2c = charsIndex.zip(chars).toMap

val data = CSV.pwd("names.txt")

/** Bookended : adds "."to either end of the string Pairs : extracts the
  * series of pairs of the bookended characters characters Ints : indexes the
  * bookended characters xenc : one hot encodes the characters excluding the
  * last character
  */
val bookended = LazyList.from(
  data
  .addColumn["Bookend", String](s => s".${s.name}.")
  .addColumn["Pairs", Seq[String]](s => s.Bookend.sliding(2).toSeq)
  .addColumn["Ints", Seq[Int]](s => s.Bookend.map(charsMap.seq))
)

println("Change lange to neural network")
println(s"Check Data")
println(bookended.take(5).mkString("\n"))

val trainData = bookended
  .flatMap(_.Pairs)
  .map(p => (first = p.head, last = p.last))
  .addColumn["target", Int](row => charsMap(row.last))
  .addColumn["xenc", Array[Double]](row => onehot(row.first, charsMap.seq))
  .toArray

trainData.take(5).mapColumn["xenc", String](_.mkString("[", ",", "]")).toSeq.ptbln

val yChars = trainData.column["target"]

val xencMall: Matrix[Double] = Matrix.fromRowsArray(
  trainData.column["xenc"].toArray
)


-- Warning: cmd2.sc:54:59 ------------------------------------------------------
54 |  .addColumn["Ints", Seq[Int]](s => s.Bookend.map(charsMap.seq))
   |                                                  ^^^^^^^^^^^^
   |method seq in trait Iterable is deprecated since 2.13.0: Iterable.seq always returns the iterable itself
-- Warning: cmd2.sc:65:70 ------------------------------------------------------
65 |  .addColumn["xenc", Array[Double]](row => onehot(row.first, charsMap.seq))
   |                                                             ^^^^^^^^^^^^
   |method seq in trait Iterable is deprecated since 2.13.0: Iterable.seq always returns the iterable itself


Change lange to neural network
Check Data
(emma,.emma.,List(.e, em, mm, ma, a.),ArraySeq(0, 5, 13, 13, 1, 0))
(olivia,.olivia.,List(.o, ol, li, iv, vi, ia, a.),ArraySeq(0, 15, 12, 9, 22, 9, 1, 0))
(ava,.ava.,List(.a, av, va, a.),ArraySeq(0, 1, 22, 1, 0))
(isabella,.isabella.,List(.i, is, sa, ab, be, el, ll, la, a.),ArraySeq(0, 9, 19, 1, 2, 5, 12, 12, 1, 0))
(sophia,.sophia.,List(.s, so, op, ph, hi, ia, a.),ArraySeq(0, 19, 15, 16, 8, 9, 1, 0))
| |first|last|target|                                                                                                         xenc|
+-+-----+----+------+-------------------------------------------------------------------------------------------------------------+
|0|    .|   e|     5|[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
|1|    e|   m|    13|[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
|2|    m|   m|    13|[0.0

import cats.syntax.all.toShow

import vecxt.all.Matrix

import vecxt.matrixUtil.row

import vecxt.all.fromRowsArray

import vecxt.BoundsCheck.DoBoundsCheck.yes


normalDist: NormalDistribution = org.apache.commons.math3.distribution.NormalDistribution@34caa160
smooth: Boolean = true
dimensions: Int = 729
defined function saveMatrixToCSV
defined function loadMatrixFromCSV
defined function onehot
generateWeights: Boolean = true
W: Matrix[Double] = vecxt.matrix$Matrix@47a157f8
chars: Vector[Char] = Vector(
  '.',
  'a',
  'b',
  'c',
  'd',
  'e',
  'f',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'q',
  'r',
  's',
  't',
  'u',
  'v',
  'w',
  'x',
  'y',
  'z'
)
charsIndex: Vector[Int] = Vector(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26
)
charsMap: Map[Char, Int] = HashMap(
  'e' -> 5,
  'n' -> 14,
  '.' -> 0,
  't' -> 20,
  'a' -> 1,
  'm' -> 13,
  '

In [ ]:
// import vecxt.all.* // this blows up the notebook for some reason
import spire.*
import spire.implicits.*
import spire.algebra.*
import vecxt.DoubleMatrix.*
import io.github.quafadas.inspireRAD.*
import vecxt.all.`*=`
import vecxt.all.@@
import vecxt.all.shape
import vecxt.all.exp
import vecxt.all.row
import vecxt.matrix.Matrix

import vecxt.all.`-=`
import vecxt.MatrixInstance.deepCopy
import vecxt.MatrixInstance.apply
import io.github.quafadas.inspireRAD.DetailShow
import io.github.quafadas.inspireRAD.LiteShow
import LiteShow.given

def heatmat(matrix: Matrix[Double], chars: Array[Char], charsIndex: Array[Int], title: String)(using LowPriorityPlotTarget): Unit | Path =
    import vecxt.BoundsCheck.DoBoundsCheck.yes
    require(chars.length == charsIndex.length, "chars and charsIndex must have the same length")

    val counts = for {
        c <- 0 until matrix.cols
    } yield
        val dist = matrix.row(c)
        EnumeratedIntegerDistribution(charsIndex, dist)

    val dists = chars.zip(counts).toMap
    val charsMap = chars.zip(charsIndex).toMap
    val i2c = charsIndex.zip(chars).toMap

    heatmap(dists, charsMap, i2c, title)

inline def calcLossF(
    weights: TejV[Matrix, Double],
    data: TejV[Matrix, Double],
    targets: Array[Int]
)(using
    inline mOps: LossContext[Matrix, Array, Scalar, Double],
    tg: TejVGraph[Double],
    fi: Fractional[Double]
): TejV[Scalar, Double] =
    val logits = data @@ weights // multiply input data by weight matrix to get raw prediction scores
    val probsNN = logits.softmaxRows // convert logits to probabilities using softmax normalization
    val range: Array[(Int, Int)] = (0 until targets.length).toArray.zip(targets) // pair each sample index with its target class
    val selected = probsNN.arrange(range) // extract the probability of the correct target class for each sample
    val theloss = (selected + 1e-8d.tej).log.mean * -1.0.tej // compute average negative log-probability (cross-entropy loss)
    theloss



@annotation.tailrec
def train(
    weights: Matrix[Double],
    data: Matrix[Double],
    targets: Array[Int],
    learningRate: Double,
    steps: Int,
    initialSteps: Int,
    initialLearningRate: Double,
    epochLength: Int = 50
): TejV[Matrix, Double] =
    given graph: TejVGraph[Double] = TejVGraph[Double]()
    val weights_ = weights.tej
    val data_ = data.tej

    if
        steps == 0 then weights_
    else
        val loss = calcLossF(weights_, data_, targets)
        val grad = loss.backward((weights = weights_))
        grad.weights *= learningRate

        val updated = weights - (grad.weights)

        if (steps - 1) % 10 == 0 then
            println(s"Step $steps, loss: ${loss.value.scalar}, rate: $learningRate")
            heatmat(updated.exp, chars.toArray, charsIndex.toArray, "NN: Coarse training")
        if steps - 1 == 0 then
            // graphDebug(graph.dag.toGraphviz)
            println(s"Final loss: ${loss.value.scalar}")
        if steps % 100 == 0 then
            println("saving weights")
            saveMatrixToCSV(updated, "weights.csv")

        val learningRate_ = if steps % epochLength == 0 then
            initialLearningRate * (1 / (1 + ((initialSteps - steps) / 20.0).toDouble ))
        else
            learningRate

        train(updated, data, targets, learningRate_, steps - 1, initialSteps, initialLearningRate)

def heatmap2(matrix: Matrix[Double], chars: Array[Char], charsIndex: Array[Int], title: String)(using LowPriorityPlotTarget): Unit | Path =
    import vecxt.BoundsCheck.DoBoundsCheck.yes
    require(chars.length == charsIndex.length, "chars and charsIndex must have the same length")

    val counts = for {
        c <- 0 until matrix.cols
    } yield
        val dist = matrix.row(c)
        EnumeratedIntegerDistribution(charsIndex, dist)

    val dists = chars.zip(counts).toMap
    val charsMap = chars.zip(charsIndex).toMap
    val i2c = charsIndex.zip(chars).toMap

    heatmap(dists, charsMap, i2c, title)


@annotation.tailrec
def train2(
    weights: Matrix[Double],
    data: Matrix[Double],
    targets: Array[Int],
    learningRate: Double,
    steps: Int,
    initialSteps: Int,
    initialLearningRate: Double,
    epochLength: Int = 50
): Matrix[Double] =
    var updated = weights.deepCopy
    if steps == 0 || steps < 0 then
        weights
    else
        for (i <- 0 to data.rows by 128) {
            given graph: TejVGraph[Double] = TejVGraph[Double]()
            val weights_ = weights.tej
            val maxIdx = Math.min(i + 500, data.rows)
            val range = Range(i, maxIdx)
            val dataSlice = data(range, ::)
            val targetsSlice = targets.slice(i, maxIdx)
            val loss = calcLossF(weights_, dataSlice.tej, targetsSlice)
            val grad = loss.backward((weights = weights_))
            grad.weights *= learningRate
            updated -= grad.weights
        }

        if (steps - 1) % 25 == 0 then
            heatmap2(updated.exp, chars.toArray, charsIndex.toArray, s"NN: Small batch update ${initialSteps - steps}")

        val learningRate_ = if steps % epochLength == 0 then
            initialLearningRate * (1 / (1 + ((initialSteps - steps) / 20.0).toDouble ))
        else
            learningRate
        // Tail recursive: last call is train
        train2(updated, data, targets, learningRate_, steps - 1, initialSteps, initialLearningRate)

println("-- Begin Training run...")

println(s"Initial weights shape: ${W.shape}")
println(s"Input data shape: ${xencMall.shape}")
println(s"Target data shape: ${yChars.length}")
println(s"First 5 target characters: ${yChars.take(5).mkString(", ")}")

val learningRate = 0.1
val steps = 51
// val trainStart = train(W, xencMall, yChars.toArray, learningRate, 2, 2, learningRate)
val weightsTrained = train2(W, xencMall, yChars.toArray, learningRate, steps, steps, learningRate)
// val weightsTrained = train(W, xencMall, yChars.toArray, learningRate, steps, steps, learningRate)


// saveMatrixToCSV(weightsTrained.value, "weights.csv")

println("training run finished")

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


-- Begin Training run...
Initial weights shape: (27,27)
Input data shape: (228146,27)
Target data shape: 228146
First 5 target characters: 5, 13, 13, 1, 0
51


Oct 31, 2025 9:14:57 PM dev.ludovic.netlib.blas.InstanceBuilder initializeNative


50
49
48
47
46
45
44
43
